In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.datasets import fetch_openml
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.kernel_approximation import Nystroem

In [ ]:
df_train = pd.read_csv('/kaggle/input/playground-series-s4e8/train.csv',header=0)
df_train.head(5)

In [ ]:
y = df_train['class']
df_train.drop(columns=['class','id'],inplace=True)
df_train.head()

In [ ]:
#For SVM and Others Classifiers is neccesary
encoder = LabelEncoder()
y = encoder.fit_transform(y)
print(y)


In [ ]:
numerical_columns = [df_train.columns[0]]
numerical_columns

In [ ]:
categorical_columns = list(df_train.columns[1:])
categorical_columns

In [ ]:
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# Combine preprocessing pipelines
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_columns),
        ('cat', categorical_transformer, categorical_columns)])

kernel = Nystroem(gamma=.2,
                random_state=1,
                n_components=300)

# Create full prediction pipeline
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('kernel',kernel),
                      ('classifier', SGDClassifier(loss="hinge", penalty="l2", max_iter=1000))])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_train, y, test_size=0.3, random_state=42)

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
y_pred = clf.predict(X_test)

In [ ]:
from sklearn.metrics import balanced_accuracy_score


In [ ]:
print("Balanced Accuracy:", balanced_accuracy_score(y_test, y_pred))

In [ ]:
df_test = pd.read_csv('/kaggle/input/playground-series-s4e8/test.csv',header=0)
df_test.head()

In [ ]:
temp = list(df_test.columns)
y_test_pred = clf.predict(df_test[temp[1:]])

In [ ]:
y_test_pred = encoder.inverse_transform(y_test_pred)
print(y_test_pred)

In [ ]:
ids = list(df_test['id'])

In [ ]:
classes = list(y_test_pred)

In [ ]:
df_solve = pd.DataFrame({
    "id":ids,
    "class":classes
})
df_solve.set_index('id',inplace=True)
df_solve.head()

In [ ]:
df_solve.to_csv('/kaggle/working/submission2.csv')